In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# load dataset for POC use
import tensorflow.keras.datasets as keras_datasets
(x_train, y_train), (x_test, y_test) = keras_datasets.cifar10.load_data()

In [3]:
# Normalize data
x_train = (x_train.astype(np.float32) - 127.5) / 127.5

# Normalize data
x_test = (x_test.astype(np.float32) - 127.5) / 127.5

In [4]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)

In [5]:
# Build a simple CNN network
from tensorflow.keras.layers import Input, Dense, concatenate, Reshape, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [6]:
def build_cnn_net(input_layer):
    x = Conv2D(32,(3,3),padding="same",activation='relu')(input_layer)
    x = Conv2D(32,(3,3),padding="same",activation='relu')(x)
    x = Conv2D(16,(5,5),padding="same",activation='relu')(x)
    x = MaxPool2D()(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(64)(x)
    x = Dense(32)(x)
    x = Dense(10,activation='softmax')(x)
    
    model = Model(input_layer, x)
    return model

input_layer = Input(shape=(32,32,3))
cnn_net = build_cnn_net(input_layer)

In [7]:
cnn_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        12816     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
__________

In [8]:
adam = Adam()
cnn_net.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Train cnn_net
cnn_net.fit(x_train,y_train_one_hot,batch_size=16, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 19s 375us/step - loss: 0.2162 - acc: 0.9165
Epoch 2/10
50000/50000 [==============================] - 12s 234us/step - loss: 0.1761 - acc: 0.9309
Epoch 3/10
50000/50000 [==============================] - 11s 224us/step - loss: 0.1625 - acc: 0.9362
Epoch 4/10
50000/50000 [==============================] - 11s 225us/step - loss: 0.1545 - acc: 0.9398
Epoch 5/10
50000/50000 [==============================] - 11s 225us/step - loss: 0.1482 - acc: 0.9423
Epoch 6/10
50000/50000 [==============================] - 11s 225us/step - loss: 0.1432 - acc: 0.9442
Epoch 7/10
50000/50000 [==============================] - 11s 225us/step - loss: 0.1395 - acc: 0.9455
Epoch 8/10
50000/50000 [==============================] - 11s 225us/step - loss: 0.1365 - acc: 0.9468
Epoch 9/10
50000/50000 [==============================] - 11s 226us/step - loss: 0.1333 - acc: 0.9481
Epoch 10/10
50000/50000 [==============================] - 11s 225us/step - loss: 

In [10]:
cnn_net.predict(x_test,verbose=1)

10000/10000 [==============================] - 1s 63us/step


array([[6.28269452e-04, 2.08878727e-03, 2.57510990e-02, ...,
        2.99948198e-03, 1.37310512e-02, 8.53412971e-03],
       [6.60155376e-04, 1.04502228e-03, 1.98956194e-08, ...,
        2.16774793e-10, 9.98293340e-01, 1.43494924e-06],
       [2.08394587e-01, 2.24277694e-02, 2.24941806e-03, ...,
        3.56814521e-03, 7.49892652e-01, 3.72021133e-03],
       ...,
       [5.48005846e-05, 8.11557754e-07, 1.26295656e-01, ...,
        7.37364888e-02, 2.46558309e-04, 3.06942020e-05],
       [1.51137486e-01, 9.02958959e-02, 1.41273551e-02, ...,
        1.18349746e-01, 1.16925105e-04, 3.90517563e-02],
       [1.72252821e-05, 2.99805879e-06, 5.53817023e-04, ...,
        6.61922932e-01, 3.68216865e-06, 1.30401440e-06]], dtype=float32)

In [11]:
import tensorflow.keras.backend as K
K.clear_session()

In [12]:
# Try OctConv
import sys
sys.path.append("OctConv-TFKeras")
from oct_conv2d import OctConv2D
# Build a simple CNN network
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD


In [13]:
# Input
input_high = layers.Input((32,32,3))
# downsampling for lower
input_low = layers.AveragePooling2D(2)(input_high)

In [14]:
def build_octconv_net(inputs,alpha):
    high, low = inputs
    
    # First OctConv
    high, low = OctConv2D(filters=32, kernel_size = (3,3),alpha=alpha)([high, low])
    #high = layers.BatchNormalization()(high)
    high = layers.Activation("relu")(high)
    #low = layers.BatchNormalization()(low)
    low = layers.Activation("relu")(low)
    # Second OctConv
    high, low = OctConv2D(filters=32, kernel_size = (3,3), alpha=alpha)([high, low])
    #high = layers.BatchNormalization()(high)
    high = layers.Activation("relu")(high)
    #low = layers.BatchNormalization()(low)
    low = layers.Activation("relu")(low)
    # Third OctConv
    #high, low = OctConv2D(filters=16, kernel_size = (5,5), alpha=alpha)([high, low])
    ##high = layers.BatchNormalization()(high)
    #high = layers.Activation("relu")(high)
    ##low = layers.BatchNormalization()(low)
    #low = layers.Activation("relu")(low)
    
    #import pdb; pdb.set_trace()
    
    # Last conv layers = alpha_out = 0 : vanila Conv2D
    # high -> high
    high_to_high = layers.Conv2D(16, (5,5), padding="same")(high)
    # low -> high
    low_to_high = layers.Conv2D(16, (5,5), padding="same")(low)
    low_to_high = layers.Lambda(lambda x: 
                        K.repeat_elements(K.repeat_elements(x, 2, axis=1), 2, axis=2))(low_to_high)
    x = layers.Add()([high_to_high, low_to_high])
    #x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    # Dense layers
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64)(x)
    x = layers.Dense(32)(x)
    x = layers.Dense(10,activation='softmax')(x)
    
    model = Model(input_high, x)
    return model

octconv_net = build_octconv_net((input_high,input_low),alpha=0.25)

In [15]:
octconv_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 16, 16, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
oct_conv2d (OctConv2D)          [(None, 32, 32, 24), 1728        input_1[0][0]                    
                                                                 average_pooling2d[0][0]          
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 24)   0           oct_conv2d[0][0]                 
__________

In [16]:
adam = Adam()
octconv_net.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Train cnn_net
octconv_net.fit(x_train,y_train_one_hot,batch_size=16, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 24s 472us/step - loss: 0.2226 - acc: 0.9145
Epoch 2/10
50000/50000 [==============================] - 21s 422us/step - loss: 0.1847 - acc: 0.9279
Epoch 3/10
50000/50000 [==============================] - 21s 419us/step - loss: 0.1693 - acc: 0.9337
Epoch 4/10
50000/50000 [==============================] - 23s 458us/step - loss: 0.1608 - acc: 0.9373
Epoch 5/10
50000/50000 [==============================] - 23s 458us/step - loss: 0.1549 - acc: 0.9397
Epoch 6/10
50000/50000 [==============================] - 24s 472us/step - loss: 0.1502 - acc: 0.9413
Epoch 7/10
50000/50000 [==============================] - 24s 472us/step - loss: 0.1455 - acc: 0.9434
Epoch 8/10
50000/50000 [==============================] - 24s 473us/step - loss: 0.1422 - acc: 0.9446
Epoch 9/10
50000/50000 [==============================] - 24s 474us/step - loss: 0.1399 - acc: 0.9454
Epoch 10/10
50000/50000 [==============================] - 24s 472us/step - loss: 

In [ ]:
# TODO: compare inference time 

In [18]:
octconv_net.predict(x_test,verbose=1)

10000/10000 [==============================] - 1s 91us/step


array([[1.7167373e-02, 2.9758541e-03, 1.0084572e-02, ..., 1.1794337e-02,
        6.5444387e-02, 1.1558613e-02],
       [4.6043918e-03, 4.6025279e-01, 1.8103879e-06, ..., 4.9362184e-06,
        5.3383464e-01, 1.2982762e-03],
       [2.0329532e-01, 2.5951472e-01, 2.0407168e-02, ..., 3.0794030e-02,
        3.9187068e-01, 7.4310154e-02],
       ...,
       [4.8434140e-05, 1.0514545e-06, 7.0177279e-02, ..., 6.4587295e-02,
        2.4248928e-05, 4.6776124e-05],
       [8.2446772e-01, 2.5925791e-02, 6.2572500e-03, ..., 3.9110673e-03,
        2.7357860e-04, 3.5498635e-04],
       [2.2097471e-05, 3.0165950e-07, 6.3765107e-04, ..., 7.0978320e-01,
        9.5099557e-07, 4.0313314e-08]], dtype=float32)